Assignment 3 is an individual assignment that will test your understanding of API calls. When we created the patent class data that you can find in our shared yandex folder we queried the API too. Your exercise is to construct the same file. You don't have to this for all the years, one year is enough. Here are the specific instructions to follow

* Pick one year X and create the patents data for that specific year. 
* Filter all patents that were granted in year X for which:
  * The first named inventor is from the USA 
  * AND the first named assignee is from the USA  
* Retrieve all the fields that we have in our class data in patent_X. 

Keep in mind: The patents API has a limit on how many patents can be displayed on one page and when there are a lot of patents you might have to retrieve multiple pages. The overall number of patents you can retrieve is 100,000 (max page number.Xmax page entries). If you expect to end up with more patents than this you have to think about a way to reduce the query by splitting the year into subsamples.

After you query the data transfer them into a data frame. Please keep in mind that you might have nested json files that you need to harmonize. 

* Save the data frame and submit the notebook to NYU classes

You can get 10 points for this assignment if you complete the entire assignment. 

In [86]:
# interacting with websites and web-APIs
import requests # easy way to interact with web sites and services
import json # read/write JavaScript Object Notation (JSON)

# data manipulation
import pandas as pd 
# normalize nested JSON files
from pandas.io.json import json_normalize

In [87]:
patent_year = 2011
per_page = 9500
max_retrieve = 100000

In [88]:
def get_patents_url(patent_year, per_page, page):
    base_url = 'https://www.patentsview.org/api/patents/query?'

    condition = '{{"_and":[{{"_eq":{{"patent_year":"{}"}}}}'\
        ',{{"_eq":{{"patent_firstnamed_inventor_country":"US"}}}}'\
        ',{{"_eq":{{"patent_firstnamed_assignee_country":"US"}}}}'\
        ']}}'.format(patent_year)

    options = '&o={{"per_page":"{}","page":"{}"}}'.format(per_page, page)
    
    fields = '&f=["patent_number","patent_title","patent_abstract","patent_num_cited_by_us_patents","patent_date",'\
        '"patent_firstnamed_inventor_city","patent_firstnamed_inventor_state","patent_firstnamed_inventor_latitude","patent_firstnamed_inventor_longitude",'\
        '"patent_firstnamed_assignee_city","patent_firstnamed_assignee_state","patent_firstnamed_assignee_latitude","patent_firstnamed_assignee_longitude",'\
        '"inventor_first_name","inventor_last_name",'\
        '"app_date","assignee_organization"'\
        ']'
    
    query = 'q={}{}{}'.format(condition, options, fields)
    return base_url + query

In [89]:
cols = ["patent_number","patent_title","patent_abstract","patent_num_cited_by_us_patents","patent_date",
        "patent_firstnamed_inventor_city","patent_firstnamed_inventor_state","patent_firstnamed_inventor_latitude","patent_firstnamed_inventor_longitude",
        "patent_firstnamed_assignee_city","patent_firstnamed_assignee_state","patent_firstnamed_assignee_latitude","patent_firstnamed_assignee_longitude"
       ]

cols3 = ["patent_number","patent_title"]

def normalize_json_response(json_response):
    if len(json_response['patents']) > 0:
        inventors_norm = pd.json_normalize(json_response['patents'], record_path='inventors', 
                                    meta=cols3).drop(["inventor_key_id"], axis=1) 
        assignees_norm = pd.json_normalize(json_response['patents'], record_path='assignees',
                                    meta=cols3).drop(["assignee_key_id"], axis=1) 
        apps_norm = pd.json_normalize(json_response['patents'], record_path='applications', 
                                    meta=cols3).drop(['app_id'], axis=1) 
        all_merged = pd.DataFrame(json_response['patents'])[cols]

        all_merged = pd.merge(all_merged, inventors_norm, how='left', left_on=cols3, right_on=cols3)
        all_merged = pd.merge(all_merged, assignees_norm, how='left', left_on=cols3, right_on=cols3)
        all_merged = pd.merge(all_merged,   apps_norm,    how='left', left_on=cols3, right_on=cols3)
        
        all_merged = all_merged.drop_duplicates(cols, keep='first')

        return all_merged
    return None

def execute_query(url):
    response = requests.get(url)
    if (response.status_code != 200):
        print(response.content, response.content)
        raise Exception(response.content)
    return response.json()

def get_patents(patent_year, per_page):
    patents = pd.DataFrame()
    page = 1
    
    url = get_patents_url(patent_year, per_page, page)
    print("Query #{}: {}".format(page, url))
    json_response = execute_query(url)
    
    total = json_response['total_patent_count']
    if total >= max_retrieve:
        pring("Warning total patents {} exceeds {}".format(total, max_retrieve))
        
    print("Total Number of Records: ", total)

    patents = patents.append(normalize_json_response(json_response))
  
    print("Number of Records collected: ",len(patents))

    while (json_response['count'] > 0) and ((len(patents)/total) < 1):
        page += 1
        url = get_patents_url(patent_year, per_page, page)
        print("Query #{}: {}".format(page, url))
        json_response = execute_query(url)
        patents = patents.append(normalize_json_response(json_response))
        print("Number of Records found: ", len(json_response['patents']), " collected so far: ", len(patents))
        
    
    return patents

In [90]:
patents_2011 = get_patents(patent_year, per_page)

Query #1: https://www.patentsview.org/api/patents/query?q={"_and":[{"_eq":{"patent_year":"2011"}},{"_eq":{"patent_firstnamed_inventor_country":"US"}},{"_eq":{"patent_firstnamed_assignee_country":"US"}}]}&o={"per_page":"9500","page":"1"}&f=["patent_number","patent_title","patent_abstract","patent_num_cited_by_us_patents","patent_date","patent_firstnamed_inventor_city","patent_firstnamed_inventor_state","patent_firstnamed_inventor_latitude","patent_firstnamed_inventor_longitude","patent_firstnamed_assignee_city","patent_firstnamed_assignee_state","patent_firstnamed_assignee_latitude","patent_firstnamed_assignee_longitude","inventor_first_name","inventor_last_name","app_date","assignee_organization"]
Total Number of Records:  99250
Number of Records collected:  9500
Query #2: https://www.patentsview.org/api/patents/query?q={"_and":[{"_eq":{"patent_year":"2011"}},{"_eq":{"patent_firstnamed_inventor_country":"US"}},{"_eq":{"patent_firstnamed_assignee_country":"US"}}]}&o={"per_page":"9500","

Number of Records found:  4250  collected so far:  99250


In [91]:
patents_2011 = patents_2011.rename(columns={
    'inventor_first_name': 'patent_firstnamed_inventor_name_first', 
    'inventor_last_name': 'patent_firstnamed_inventor_name_last',
    'assignee_organization': 'patent_firstnamed_assignee_organization'
})

In [92]:
cols_orig = ["patent_number","patent_title","patent_abstract","patent_num_cited_by_us_patents","patent_date",
             "app_date","patent_firstnamed_inventor_name_first","patent_firstnamed_inventor_name_last",
             "patent_firstnamed_inventor_city","patent_firstnamed_inventor_state","patent_firstnamed_inventor_latitude",
             "patent_firstnamed_inventor_longitude","patent_firstnamed_assignee_organization",
             "patent_firstnamed_assignee_city","patent_firstnamed_assignee_state","patent_firstnamed_assignee_latitude",
             "patent_firstnamed_assignee_longitude"
]
patents_2011 = patents_2011[cols_orig]

In [93]:
patents_2011

,patent_number,patent_title,patent_abstract,patent_num_cited_by_us_patents,patent_date,app_date,patent_firstnamed_inventor_name_first,patent_firstnamed_inventor_name_last,patent_firstnamed_inventor_city,patent_firstnamed_inventor_state,patent_firstnamed_inventor_latitude,patent_firstnamed_inventor_longitude,patent_firstnamed_assignee_organization,patent_firstnamed_assignee_city,patent_firstnamed_assignee_state,patent_firstnamed_assignee_latitude,patent_firstnamed_assignee_longitude
0,7861318,Pad wearable over articulated joint,"A pad, such as a knee, elbow, or shoulder pad,...",0,2011-01-04,2007-01-29,William L.,"Grilliot, Jr.",Dayton,OH,39.7589,-84.1917,"Morning Pride Manufacturing, L.L.C.",Dayton,OH,39.7589,-84.1917
3,7861319,Garment with enhanced knee support,The present invention relates to a garment wit...,30,2011-01-04,2009-03-05,Michael Ray,Torry,Edwards,CO,39.645,-106.594,"AlignMed, Inc.",Scottsdale,AZ,33.5092,-111.898
4,7861321,Sports glove having protective knuckle segment,A protective sports glove that includes a prot...,9,2011-01-04,2008-03-19,Matthew M.,Winningham,Royal Oak,MI,42.4894,-83.1447,"Warrior Sports, Inc.",Warren,MI,42.4775,-83.0278
5,7861330,Universal toilet tank lever,A toilet tank lever assembly may be mounted in...,3,2011-01-04,2006-04-17,Iosif,Manga,Charlotte,NC,35.2269,-80.8433,"Brasstech, Inc.",Santa Ana,CA,33.7456,-117.867
8,7861331,Reduced water consumption flush toilet,A reduced water consumption flush toilet for a...,8,2011-01-04,2003-11-26,George,Grech,Garden City,MI,42.3256,-83.3311,Thetford Corporation,Ann Arbor,MI,42.2708,-83.7264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10543,RE43045,Multi-chamber MOCVD growth apparatus for high ...,In one embodiment the present invention is a m...,0,2011-12-27,2010-05-05,David,Emerson,Chapel Hill,NC,35.9131,-79.0561,"Cree, Inc.",Durham,NC,35.9939,-78.8989
10544,RE43047,MMS based photo album publishing system,An MMS publishing system comprises a managemen...,0,2011-12-27,2009-11-24,Vinod V.,Valloppillil,Foster City,CA,37.5586,-122.27,Openwave Systems Inc.,Redwood City,CA,37.4853,-122.235
10545,RE43048,Ultrasonic diagnostic imaging with harmonic co...,Apparatus and methods are disclosed for the de...,0,2011-12-27,2000-01-11,Juin-Jet Jet,Hwang,Mercer Island,WA,47.5708,-122.221,"Advanced Technology Laboratories, Inc.",Bothell,WA,47.7625,-122.204
10547,RE43049,Rigidly-linked articulating wrist with decoupl...,The present invention is a device having a rig...,19,2011-12-27,2002-06-07,Kenneth Wayne,Grace,Goleta,CA,34.4358,-119.827,"Intuitive Surgical Operations, Inc.",Sunnyvale,CA,37.3689,-122.035
